Introduction à Rust

# Gestion de la mémoire en Rust 🦀🦀


[Pierre-Antoine Champin](http://champin.net/)

Département Info Doua – [IUT Lyon 1](http://iut.univ-lyon1.fr/)

## Rappel : la pile et le tas

La pile

- contient les données des variables,
- de taille connue à la compilation (statique),
- gérées de manière *Last in, first out* par le compilateur (quel que soit le langage)
    
Le tas

- contient des données de taille connue à l'exécution (dynamique),
- gérées différemment selon les langages.

### Gestion du tas

En C
- le développeur a la responsabilité de libérer la mémoire,
- ce qui peut donner lieu à des erreurs (mémoire libérée trop tôt, trop tard, jamais...).

En Java
- le ramasse-miettes (*garbage collector*) se charge de libérer la mémoire à l'exécution,
- ce qui peut entraîner des problèmes de performances.

### Gestion du tas en Rust

En Rust
- le **compilateur** se charge de libérer la mémoire.
- Cela suppose de lui fournir assez d'information pour qu'il le fasse,
- d'où les concepts spécifiques de **propriété** et d'**emprunt**. 🦀🦀

## La sémantique *move*

Dans la plupart des langages de programmation,
l'affectation a une sémantique de copie.

```c
v1 = init();
v2 = v1;
// v1 et v2 contiennent « la même chose »
```

En Rust, l'affectation a une semantique de *déplacement* (*move*):

* conceptuellement, l'affectation *déplace* la valeur de `v1` dans `v2`.
* après l'affectation, `v1` est considérée comme non-initialisée.


### Illustration : le type `Vec`

In [2]:
let v1: Vec<i32> = vec![22, 44, 66];

<div style="text-align: center">
<img alt="String en mémoire" src="images/trpl04-01.svg" style="display: inline; width: 12em;">
</div>

### Avec la sémantique *copy* du C

```c
Vec v1 = init_vec();
Vec v2 = v1;
```

<div style="text-align: center"> <img alt="Shallow copy" src="images/trpl04-02.svg" style="display: inline; width: 12em;"> </div>

### Avec la sémantique *move* de Rust 🦀

In [3]:
let v1: Vec<i32> = vec![22, 44, 66];
println!("v1={:?}", &v1);
let v2 = v1;
println!("v2={:?}", &v2);
//println!("v1={:?}", &v1); // NE COMPILE PAS

v1=[22, 44, 66]
v2=[22, 44, 66]


<div style="text-align: center">
<img alt="String en mémoire" src="images/trpl04-04.svg" style="display: inline; width: 12em;">
</div>

### Copie (profonde) explicite

In [4]:
// copie profonde EXPLICITE avec la méthode clone
let v1: Vec<i32> = vec![22, 44, 66];
let v2 = v1.clone();
println!("v1={:?} v2={:?}", &v1, &v2);

v1=[22, 44, 66] v2=[22, 44, 66]


<div style="text-align: center">
<img alt="Copie profonde" src="images/trpl04-03.svg" style="display: inline; width: 12em;">
</div>

### Copie implicite

Bien que la sémantique *move* soit la règle,
il existe des exceptions.

En particulier, les affectations pour tous les types de base (entiers, flottants, booléens...)
ont la sémantique *copy*,
car le compilateur sait que cette copie est sûre (*safe*).

In [5]:
let x = 42;
let y = x; // copy instead of move
println!("x={} y={}", x, y);

x=42 y=42


Plus généralement,
tout type implémentant le [trait `Copy`](https://doc.rust-lang.org/stable/std/marker/trait.Copy.html)
utilise la sémantique *copy*.

Il est possible de faite implémenter `Copy` aux types définis par nous, mais ce n'est pas toujours possible ni judicieux (example : le type `Vec`).

## Propriété (*ownership*) 🦀🦀

* Toute donnée est la **propriété** d'exactement une variable (ou champ de structure).

* Conceptuellement, l'affectation d'une variable à une autre **déplace** (*move*) les données,
  et **transfère** la propriété à la nouvelle variable.
  
* Lorsqu'une variable disparait,
  (i.e. lorsqu'on quitte son scope),
  si elle est encore propriétaire de ses données,
  celles-ci sont **libérées**.

In [6]:
fn fibo(nb: usize) -> Vec<i32> {
    let mut v1 = vec![];                  // 1. Le vecteur est créé ici
    for i in 0..nb {
        if i < 2 { v1.push(1); }
        else     { v1.push(v1[i-2] + v1[i-1]); }
    }
    v1                                    // 2. Sa propriété est transmise au code appelent
}

fn somme(v2: Vec<i32>) -> i32 {
    v2.into_iter().sum()
}                                         // 5. La variable 'v' arrive au bout de son scope,
                                          //    et elle possède toujours le vecteur,
                                          //    qui est donc libérée.

{ 
    let n = 7;
    let mut v3 = fibo(n);                 // 3. Le vecteur est récupérée depuis la fonc. fibo
    println!("v3={:?}", &v3);
    let s = somme(v3);                    // 4. Le vecteur est transférée à la fonc. somme
    println!("la somme des {} premiers termes vaut {}", n, s);
    // println!("{:?}", v3); // NE COMPILE PAS
};

v3=[1, 1, 2, 3, 5, 8, 13]
la somme des 7 premiers termes vaut 33


## Emprunts (*borrow*) 🦀🦀

In [7]:
let v1: Vec<i32> = vec![22, 44, 66];
{
    let v2 = &v1;
    println!("v1={:?} v2={:?}", &v1, &v2);
}
println!("v1={:?}", &v1);

v1=[22, 44, 66] v2=[22, 44, 66]
v1=[22, 44, 66]


<div style="text-align: center">
<img alt="String en mémoire" src="images/trpl04-05.svg" style="display: inline; width: 16em;">
</div>

Pour tout type `T`, les variables de type `&T` sont des **références** vers un `T`.

* Physiquement, les références sont des pointeurs.
* Conceptuellement, les références ne *possèdent pas* les données vers lesquelles elles pointent,
* La disparition de la référence n'entraîne donc pas la libération des données.
* En conséquence, plusieurs références vers les même données peuvent *co-exister*.

In [8]:
let mut v1: Vec<i32> = vec![22, 44, 66];
{
    let v2 = &v1;
    let v3 = &v1;
    println!("v1={:?} v2={:?} v3={:?}", &v1, &v2, &v3);
}
println!("v1={:?}", &v1);

v1=[22, 44, 66] v2=[22, 44, 66] v3=[22, 44, 66]
v1=[22, 44, 66]


NB : Les références de type `&T` sont immutables.

In [9]:
let mut v1: Vec<i32> = vec![22];
v1.push(44);
{
    let v2 = &v1;
    //v2.push(66); // NE COMPILE PAS
    println!("v1={:?} v2={:?}", &v1, &v2);
}
v1.push(88);
println!("v1={:?}", &v1);

v1=[22, 44] v2=[22, 44]
v1=[22, 44, 88]


Pour que l'exemple ci-dessus fonctionne, il faut utiliser une **référence mutable**,
de type `&mut T`.

In [10]:
let mut v1: Vec<i32> = vec![22];
v1.push(44);
{
    let v2 = &mut v1;
    v2.push(66);
}
v1.push(88);
println!("v1={:?}", &v1);

v1=[22, 44, 66, 88]


### Règles sur les références

On ne peut pas avoir en même temps

* plusieurs références mutables, ni
* une référence mutable et une (ou plusieurs) référence(s) immutable(s).

En effet, il faut éviter

* que deux parties du code modifient les mêmes données simultanément, et
* qu'une partie du code lise des données en cours de modification.

In [11]:
let mut v1: Vec<i32> = vec![22];
v1.push(44);
{
    let v2 = &mut v1;
    //v1.push(66); // NE COMPILE PAS
    //let v3 = &v1; // NE COMPILE PAS
    println!("v2={:?}", &v2);
}
v1.push(88);
println!("v1={:?}", &v1);

v2=[22, 44]
v1=[22, 44, 88]


### Différence conceptuelle entre référence et pointeur


In [12]:
let a = 42;
let b = 41 + 1;
let test = (&a == &b);
test

true

* La « valeur » d'une référence n'est **pas** l'adresse en mémoire, c'est la valeur pointée.

    * Ne pas penser `&T` comme « un pointeur vers un `T` » mais comme « un `T` emprunté à quelqu'un d'autre ».
    * `T` et `&T` restent malgré tout deux types différents...

* En Rust, une référence n'est **jamais** un pointeur null. 🦀

## Choix des types des paramètres des fonctions

In [13]:
// ma fonction n'a pas besoin de modifier les données : &T
fn compte_zeros(v: &Vec<i32>) -> usize {
    let mut c = 0;
    for i in v {
        if *i == 0 {
            c += 1;
        }
    }
    c
}
// NB: cette version n'est pas très idiomatique...

In [14]:
// ma fonction a besoin de modifier les données : &mut T
fn ajoute_un(v: &mut Vec<i32>) {
    for i in v {
        *i += 1;
    }
}

In [15]:
// ma fonction "consomme" les données (rare) / les données sont `Copy` : T
fn fact(n: usize) -> usize {
    if n <= 1 { 1 } else { n*fact(n-1) }
}

## Durées de vie (*lifetime*)

Un composant du compilateur (le *borrow checker*)
vérifie qu'aucun emprunt ne dure plus longtemps que la durée de vie du propriétaire.

(puisqu'à la « mort » du propriétaire, les données sont libérées)

In [16]:
// NE COMPILE PAS
{
    let mut emprunteur: Vec<&i32> = Vec::new();
    {
        let proprietaire = 42;
        emprunteur.push(&proprietaire)
    }
    println!("{:?}", &emprunteur);
}

Error: `proprietaire` does not live long enough

Dans certaines situations, le compilateur a besoin qu'on lui donne des indications sur les durées de vie respectives des différentes références (« qui doit vivre aussi longtemps que qui ? »).

In [17]:
fn borrow_concat(v1: &Vec<i32>, v2: &Vec<i32>) -> Vec<&i32> {
    // indique que v1 et v2 doivent tous deux vivre aussi longtemps que le vecteur retourné
    let mut v3 = Vec::with_capacity(v1.len() + v2.len());
    for i in 0..v1.len() {
        v3.push(&v1[i]);
    }
    for i in 0..v2.len() {
        v3.push(&v2[i]);
    }
    v3
}
// NB: cette version n'est pas très idiomatique...

Error: missing lifetime specifier

Solution du problème précédent : on introduit des identifiants de *lifetime*.

(En l'occurrence, un seul, arbitrairement nommé `'a`,
 pour indiquer que les trois types de références doivent vivre aussi longtemps les uns que les autres.)

In [18]:
fn borrow_concat<'a>(v1: &'a Vec<i32>, v2: &'a Vec<i32>) -> Vec<&'a i32> {
    // indique que v1 et v2 doivent tous deux vivre aussi longtemps que
    // les éléments du vecteur retourné
    let mut v3 = Vec::with_capacity(v1.len() + v2.len());
    for i in 0..v1.len() {
        v3.push(&v1[i]);
    }
    for i in 0..v2.len() {
        v3.push(&v2[i]);
    }
    v3
}
// NB: cette version n'est pas très idiomatique...

Il existe des règles par défaut pour les *lifetimes*,
qui font que dans de nombreux cas, il n'est pas nécessaire de les spécifier.

Par exemple, s'il n'y a qu'une seule référence en entrée, et qu'une seule référence en sortie,
elles sont supposées avoir la même *lifetime*.

In [19]:
fn borrow_concat(v1: &Vec<i32>) -> Vec<&i32> {
// équivalent à
// fn borrow_concat<'a>(v1: &'a Vec<i32>) -> Vec<&'a i32> {
    let mut v2 = Vec::with_capacity(v1.len());
    for i in 0..v1.len() {
        v2.push(&v1[i]);
    }
    v2
}
// NB: cette version n'est pas très idiomatique...